# 1. Getting Started

multiTFA is a python package for accurate thermodynamic flux analysis. multiTFA is built on COBRApy to take advantage of COBRA model structure. For installation instructions please refer to INSTALL.rst.

## 1.1. Building a multiTFA model

multiTFA models can be readily constructed from COBRA model objects. To create a model, we require 

1. A COBRA model

2. Metabolite KEGG/SEED identifiers - This is required to match and retrieve thermodyanamic information against our database

3. Compartment specific pH and ionic strength information. To calculate Gibbs energy related to transmembrane transport, information about membrane potential is also required.

Additionally, for accurate thermodynamic calculations, intracellular metabolite concentrations can be provided. 

**NOTE**: All the input information should be in S.I units, e.g: ionic strength, metabolite concentration in Moles.

To demonstrate the usage, we use reduced *E. Coli* model borrowed from pyTFA repository.


In [6]:
from cobra import io
from multiTFA.core.tmodel import tmodel

# Load COBRA model
cobra_model = io.load_matlab_model('multiTFA/model/small_ecoli.mat')

# Define dictionary of metabolite identifiers
Kegg_map = {}
with open("multiTFA/model/ecoli_kegg_map.txt", "r") as f:
    for line in f:
        line = line.strip()
        line = line.split("\t")
        Kegg_map[line[0]] = line[1]

# compartment specific properties
pH_I_dict = {
    "pH": {"c": 7.5, "e": 7, "p": 7},
    "I": {"c": 0.25, "e": 0, "p": 0},
}

# Membrane potentials
del_psi_dict = {
    "c": {"e": 0, "p": 150},
    "e": {"c": 0, "p": 0},
    "p": {"c": -150, "e": 0},
}

# List of reaction ids that you would like to exclude from thermodynamic analysis, here we exclude demand reactions
Excl = [i.id for i in cobra_model.reactions if "DM_" in i.id]

# Optionally, define metabolite concentration bounds
conc_dict = {
    "min": {"atp_c": 1e-3, "adp_c": 4e-4, "amp_c": 2e-4},
    "max": {"atp_c": 1e-2, "adp_c": 7e-4, "amp_c": 3e-4},
}

# Now build the multiTFA model from COBRA model
t_model = tmodel(
    model=cobra_model,
    Kegg_map=Kegg_map,
    pH_I_dict=pH_I_dict,
    del_psi_dict=del_psi_dict,
    Exclude_list=Excl,
    concentration_dict=conc_dict,
)

print(t_model)

Read LP format model from file /tmp/tmpl2z8bpgs.lp
Reading time = 0.01 seconds
: 304 rows, 1198 columns, 12812 nonzeros
model_red


## 1.2 Metabolites

Reactions and metabolites in multiTFA model can be accessed similar to a COBRA model. For demonstration, we will consider cytosolic 'ATP' as our metabolite with 'atp_c' identifier in our model. Let us access 'ATP' and print it's standard Gibbs energy of formation. Also, change its intracellular concentration lower and upper bounds to '2e-3' and '5e-2' M respectively. These changes will reflect in solver variables and constraints.

In [9]:
atp = t_model.metabolites.get_by_id('atp_c')
print("Standard Gibbs energy of formation of ATP is {} KJ/mol".format(atp.delG_f))
print("initial concentrations")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))
atp.concentration_min = 2e-3
atp.concentration_max = 5e-2
print("Concentrations after modifying")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))

Standard Gibbs energy of formation of ATP is -2814.238797234083 KJ/mol
initial concentrations
atp_c	0.002	0.05
Concentrations after modifying
atp_c	0.002	0.05


## 1.3 Reactions

Reactions in multiTFA model are of type cobra.DictList. Similar to COBRA model, reactions can be accessed/retrieved by its index or by it's name. Let us consider ATP synthase raction in our model by 'ATPS4rpp' and access it's attributes.


In [10]:
atps = t_model.reactions.get_by_id('ATPS4rpp')
print(atps.reaction)

adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c


We can check if the reaction is involved in any transmembrane transport

In [11]:
print(atps.isTrans)

True


Also, we can check which metabolites are involved in transport

In [12]:
print(atps.find_transportMets())

{<Thermo_met h_p at 0x7fade6813cf8>: <Thermo_met h_c at 0x7fade6487f98>}


Finally, we can access the Gibbs energy of transport or transformed Gibbs energy of the reaction

In [14]:
print("Gibbs energy of transport is {} KJ/mol".format(atps.transport_delG))
print("Transform Gibbs energy of is {} KJ/mol".format(atps.delG_transform))

Gibbs energy of transport is -57.891 KJ/mol
Transform Gibbs energy of is -29.838045425074974 KJ/mol


Note: Transport Gibbs energy is included in transform Gibbs energy of a reaction